# Введение 

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Загрузка и подготовка данных

## Импорты

In [1]:
import numpy as np
import pandas as pd
import re
import torch
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score, classification_report
import optuna
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
from tqdm import tqdm
import warnings

C:\Users\Deвайс\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Загрузка необходимых ресурсов NLTK
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Deвайс\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Deвайс\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Deвайс\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Deвайс\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используем устройство: {device}")

Используем устройство: cuda


## 2. Подготовка

Загрузим данные и ознакомимся с их структурой. В датасете есть два столбца: text (текст комментария) и toxic (целевой признак, где 1 — токсичный комментарий, 0 — нет).

In [4]:
# Загрузка данных
file_path = '/Users/Deвайс/ML/toxic_comments.csv'  # Правильный путь к файлу
df = pd.read_csv(file_path)

In [5]:
# Проверка данных
display(df.head())
print('-------------------------------------------------------------------')
print(df.info())
print('-------------------------------------------------------------------')
print(df['toxic'].value_counts(normalize=True))  # Проверяем баланс классов

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


-------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
None
-------------------------------------------------------------------
toxic
0    0.898388
1    0.101612
Name: proportion, dtype: float64


**Примечания:**
- **Количество строк в датасете**: 159,292.
- **Типы данных**:
    - `Unnamed: 0`: целочисленный тип (идентификатор).
    - `text`: строковый тип (текст комментария).
    - `toxic`: целочисленный тип (метка токсичности).

**Статистика по целевому признаку `toxic`:**
- 89.8% комментариев не токсичные (0).
- 10.2% комментариев токсичные (1).


In [6]:
# Проверим на наличие пропущенных значений
print("Количество пропущенных значений:\n", df.isna().sum())

Количество пропущенных значений:
 Unnamed: 0    0
text          0
toxic         0
dtype: int64


In [7]:
# Удалим дубликаты, если они есть
df = df.drop_duplicates().reset_index(drop=True)

## 3. Предобработка текста

На этом этапе мы выполняем предобработку текстовых данных, чтобы привести их в форму, пригодную для обучения модели.

In [8]:
# Модули для предобработки текста
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [9]:
# Функция предобработки текста
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    text = re.sub(r'<.*?>', ' ', text)  # Удаление HTML-тегов
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)  # Удаление ссылок
    text = re.sub(r'\s+', ' ', text).strip()  # Удаление лишних пробелов
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [10]:
# Применяем предобработку
df['text'] = df['text'].apply(preprocess_text)

**Шаги предобработки:**
1. **Приведение текста к нижнему регистру** — для унификации.
2. **Удаление всех символов, не являющихся буквами и цифрами** — с помощью регулярных выражений.
3. **Удаление HTML-тегов** — чтобы избавиться от ненужной разметки.
4. **Удаление ссылок** — с помощью регулярных выражений.
5. **Удаление лишних пробелов** — для очистки текста.
6. **Токенизация** — разбиение текста на отдельные слова.
7. **Лемматизация** — приведение слов к их базовой форме.
8. **Удаление стоп-слов** — таких как "the", "and", которые не несут значимой информации для классификации.

**Применение предобработки:**
Мы применяем эту функцию ко всем строкам в столбце `text` с помощью метода `apply()`.

```python
# Пример применения предобработки
df['text'] = df['text'].apply(preprocess_text)


In [31]:
#check
df

,Unnamed: 0,text,toxic
0,0,explanation edits made username hardcore metal...,0
1,1,aww match background colour seemingly stuck th...,0
2,2,hey man really trying edit war guy constantly ...,0
3,3,make real suggestion improvement wondered sect...,0
4,4,sir hero chance remember page,0
...,...,...,...
159287,159446,second time asking view completely contradicts...,0
159288,159447,ashamed horrible thing put talk page 128 61 19 93,0
159289,159448,spitzer umm there actual article prostitution ...,0
159290,159449,look like actually put speedy first version de...,0


Текст предобработан перейдем к вычислениям. 

## 4. Разделение данных и векторизация

### 4.1 Разделение данных

Мы разделяем данные на обучающую и тестовую выборки с помощью функции `train_test_split()` из библиотеки `sklearn`. Размер тестовой выборки составляет 20% от всех данных. Мы также используем параметр `stratify`, чтобы гарантировать, что распределение классов в обучающей и тестовой выборках будет одинаковым.

In [11]:
# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['toxic'], test_size=0.2, random_state=42, stratify=df['toxic'])

### 4.2 TF-IDF векторизация + Optuna
Для представления текста в числовой форме мы используем метод TF-IDF векторизации (Term Frequency - Inverse Document Frequency). Это помогает учесть важность каждого слова в контексте всего корпуса данных.

Мы ограничиваем количество признаков до 20,000 и используем диграммы (пары слов), чтобы лучше захватывать контекст.

In [12]:
# TF-IDF векторизация
vectorizer = TfidfVectorizer(max_features=20000, ngram_range=(1,2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [13]:
# Отключаем FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Устанавливаем стандартный уровень логирования Optuna
optuna.logging.set_verbosity(optuna.logging.INFO)

### 4.3 Оптимизация моделей с Optuna

Для выбора наилучшей модели и ее гиперпараметров мы используем библиотеку **Optuna**, которая позволяет автоматически искать оптимальные параметры для машинного обучения.

**Поиск наилучшей модели**
Мы рассматриваем три различных модели классификации:
- **Логистическая регрессия (`logreg`)**
- **Случайный лес (`random_forest`)**
- **Градиентный бустинг LightGBM (`lightgbm`)**

Optuna будет **перебирать** параметры для каждой модели, чтобы найти наилучшие.

In [14]:
# Функция для оптимизации моделей с Optuna
def objective(trial):
    model_type = trial.suggest_categorical('model_type', ['logreg', 'random_forest', 'lightgbm'])
    
    if model_type == 'logreg':
        C = trial.suggest_float('C', 1e-3, 10, log=True)
        model = LogisticRegression(class_weight='balanced', max_iter=1000, C=C)
    elif model_type == 'random_forest':
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 15)
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, class_weight='balanced', random_state=42)
    else:
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 15)
        model = LGBMClassifier(n_estimators=n_estimators, max_depth=max_depth, class_weight='balanced', random_state=42, device='gpu', verbose=-1)
    
    cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train_tfidf, y_train, cv=cv, scoring='f1')

    return scores.mean()

In [15]:
# Создаем исследование и запускаем оптимизацию
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2025-02-09 16:53:10,884] A new study created in memory with name: no-name-ca4da2d0-5a1c-4ea7-97c8-65b9e92c9eb1
[I 2025-02-09 16:53:11,714] Trial 0 finished with value: 0.7057534362314926 and parameters: {'model_type': 'logreg', 'C': 0.08659934409452559}. Best is trial 0 with value: 0.7057534362314926.
[I 2025-02-09 16:53:34,858] Trial 1 finished with value: 0.7286330277955044 and parameters: {'model_type': 'lightgbm', 'n_estimators': 221, 'max_depth': 4}. Best is trial 1 with value: 0.7286330277955044.
[I 2025-02-09 16:54:16,036] Trial 2 finished with value: 0.40796670652529066 and parameters: {'model_type': 'random_forest', 'n_estimators': 250, 'max_depth': 10}. Best is trial 1 with value: 0.7286330277955044.
[I 2025-02-09 16:54:16,984] Trial 3 finished with value: 0.7391086840626943 and parameters: {'model_type': 'logreg', 'C': 5.471406448047998}. Best is trial 3 with value: 0.7391086840626943.
[I 2025-02-09 16:54:27,304] Trial 4 finished with value: 0.3815692532122098 and paramet

In [33]:
# Получаем лучшую попытку из Optuna
best_trial = study.best_trial

# Выводим лучшую модель и ее параметры
print(f"Лучший F1-score: {best_trial.value:.4f}")
print(f"Лучшая модель: {best_trial.params['model_type']}")
print("Лучшие параметры:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")


Лучший F1-score: 0.7626
Лучшая модель: lightgbm
Лучшие параметры:
  model_type: lightgbm
  n_estimators: 300
  max_depth: 14


Лучшая модель: В ходе оптимизации гиперпараметров было выявлено, что лучшая модель для данной задачи — это LightGBM. С результатом F1-score = 0.7626 на тестовой выборке. Используем технологию BERT для улучшения качества идентификации токсичных комментариев.

## 5. Использование BERT для преобразования текста в эмбеддинги

Для улучшения качества классификации мы используем предобученную нейросетевую модель **BERT** (`unitary/toxic-bert`). Эта модель была специально обучена на токсичных комментариях, что делает ее подходящей для нашей задачи.

**Загрузка модели и токенизатора BERT**

Мы загружаем:
- **Токенизатор** `BertTokenizer` — для преобразования текста в числовые представления (токены).
- **Модель** `BertForSequenceClassification` — BERT, адаптированный для задачи классификации текста.

In [16]:
# Загрузка модели и токенизатора BERT
tokenizer = BertTokenizer.from_pretrained('unitary/toxic-bert')
model_bert = BertForSequenceClassification.from_pretrained('unitary/toxic-bert').to(device)

In [17]:
# Функция преобразования текста в эмбеддинги BERT
def get_bert_embeddings(texts, tokenizer, model, max_length=128):
    model.eval()
    embeddings = []
    with torch.no_grad():
        for text in texts:
            encoding = tokenizer(text, padding='max_length', truncation=True, max_length=max_length, return_tensors="pt").to(device)
            output = model(**encoding)
            embeddings.append(output.logits.cpu().numpy())
    return np.vstack(embeddings)

In [18]:
# Преобразуем текст в эмбеддинги
X_train_bert = get_bert_embeddings(X_train.tolist(), tokenizer, model_bert)
X_test_bert = get_bert_embeddings(X_test.tolist(), tokenizer, model_bert)

### 5.1 Оптимизация моделей с Optuna + BERT

Для выбора наилучшей модели и ее гиперпараметров мы используем библиотеку **Optuna**, которая позволяет автоматически искать оптимальные параметры для машинного обучения.

**Поиск наилучшей модели**

Мы рассматриваем три различных модели классификации:
- **Логистическая регрессия (`logreg`)**
- **Случайный лес (`random_forest`)**
- **Градиентный бустинг LightGBM (`lightgbm`)**

Optuna будет **перебирать** параметры для каждой модели, чтобы найти наилучшие.

In [19]:
# Оптимизация модели с BERT эмбеддингами
def objective_bert(trial):
    model_type = trial.suggest_categorical('model_type', ['logreg', 'random_forest', 'lightgbm'])
    
    if model_type == 'logreg':
        C = trial.suggest_float('C', 1e-3, 10, log=True)
        model = LogisticRegression(class_weight='balanced', max_iter=1000, C=C)
    elif model_type == 'random_forest':
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 15)
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, class_weight='balanced', random_state=42)
    else:
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 15)
        model = LGBMClassifier(n_estimators=n_estimators, max_depth=max_depth, class_weight='balanced', random_state=42, device='gpu', verbose=-1)

    # Кросс-валидация для оценки модели
    cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train_bert, y_train, cv=cv, scoring='f1')

    return scores.mean()

In [20]:
# Создаем исследование и запускаем оптимизацию
study_bert = optuna.create_study(direction='maximize')
study_bert.optimize(objective_bert, n_trials=50)
print("Оптимизация моделей с BERT эмбеддингами завершена!")

[I 2025-02-09 17:37:55,548] A new study created in memory with name: no-name-96db4961-2d9c-4138-9faa-d36a8cfd4f37
[I 2025-02-09 17:37:59,682] Trial 0 finished with value: 0.7940937803502945 and parameters: {'model_type': 'lightgbm', 'n_estimators': 237, 'max_depth': 14}. Best is trial 0 with value: 0.7940937803502945.
[I 2025-02-09 17:40:10,123] Trial 1 finished with value: 0.8195845738636827 and parameters: {'model_type': 'random_forest', 'n_estimators': 272, 'max_depth': 11}. Best is trial 1 with value: 0.8195845738636827.
[I 2025-02-09 17:42:16,304] Trial 2 finished with value: 0.8247564780494955 and parameters: {'model_type': 'random_forest', 'n_estimators': 253, 'max_depth': 12}. Best is trial 2 with value: 0.8247564780494955.
[I 2025-02-09 17:42:17,635] Trial 3 finished with value: 0.783852513829616 and parameters: {'model_type': 'lightgbm', 'n_estimators': 214, 'max_depth': 3}. Best is trial 2 with value: 0.8247564780494955.
[I 2025-02-09 17:42:18,110] Trial 4 finished with valu

Оптимизация моделей с BERT эмбеддингами завершена!


In [21]:
# Загружаем лучшие гиперпараметры и обучаем финальную модель
trial_bert = study_bert.best_trial
best_model_type = trial_bert.params['model_type']

if best_model_type == 'logreg':
    best_C = trial_bert.params['C']
    model = LogisticRegression(class_weight='balanced', max_iter=1000, C=best_C)
elif best_model_type == 'random_forest':
    best_n_estimators = trial_bert.params['n_estimators']
    best_max_depth = trial_bert.params['max_depth']
    model = RandomForestClassifier(n_estimators=best_n_estimators, max_depth=best_max_depth, class_weight='balanced', random_state=42)
else:
    best_n_estimators = trial_bert.params['n_estimators']
    best_max_depth = trial_bert.params['max_depth']
    model = LGBMClassifier(n_estimators=best_n_estimators, max_depth=best_max_depth, class_weight='balanced', random_state=42, device='gpu')

In [29]:
# Определяем тип лучшей модели
best_model_type = trial_bert.params['model_type']

# Выводим информацию о лучшей модели и ее параметрах
print(f"✅ Лучшая модель: {best_model_type}")
print(f"🎯 Лучший F1-score: {trial_bert.value:.4f}")
print("⚙️ Лучшие гиперпараметры:")
for key, value in trial_bert.params.items():
    print(f"  {key}: {value}")

✅ Лучшая модель: random_forest
🎯 Лучший F1-score: 0.8379
⚙️ Лучшие гиперпараметры:
  model_type: random_forest
  n_estimators: 63
  max_depth: 15


## 5.2 Обучение на тестовых данных лучшей модели

In [22]:
# Обучаем модель на тренировочных данных с BERT эмбеддингами
model.fit(X_train_bert, y_train)

# Предсказания на тестовых данных
y_pred_bert = model.predict(X_test_bert)

# Выводим метрики качества на тестовой выборке
print("Результаты на тестовых данных:")
print(classification_report(y_test, y_pred_bert))

Результаты на тестовых данных:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98     28622
           1       0.79      0.89      0.84      3237

    accuracy                           0.96     31859
   macro avg       0.89      0.93      0.91     31859
weighted avg       0.97      0.96      0.97     31859



In [27]:
# Вычисляем F1-score
f1 = f1_score(y_test, y_pred_bert, average='weighted')

#F1-score
print(f"F1-score: {f1:.4f}")

F1-score: 0.9652


### 6. Общий вывод по проекту:
**1. Цель проекта:**
Проект направлен на классификацию токсичных комментариев с использованием различных моделей машинного обучения, включая методы на основе BERT эмбеддингов. Мы использовали инструменты для оптимизации гиперпараметров моделей с помощью Optuna и достижения наилучших результатов.

**2. Процесс и используемые технологии:**
Предобработка данных: Мы использовали различные методы предобработки текста, такие как удаление стоп-слов, приведение к нижнему регистру и лемматизация для очистки текста.
TF-IDF и BERT эмбеддинги: Для векторизации текста использовались как традиционные методы (TF-IDF), так и более современные BERT эмбеддинги, которые дали значительное улучшение в точности модели.
Оптимизация гиперпараметров: Оптимизация моделей проводилась с использованием библиотеки Optuna, что позволило найти лучшие параметры для каждой модели.
Использование GPU: Для обучения моделей использовался GPU, что значительно ускорило процесс при работе с большими объемами данных и моделями на основе BERT.

**3. Результаты моделей:**
Лучшая модель: В ходе оптимизации гиперпараметров было выявлено, что лучшая модель для данной задачи — это Random Forest. С результатом F1-score = 0.8379 на тестовой выборке.
Параметры лучшей модели:
n_estimators: 63
max_depth: 15
Метрики качества на тестовых данных:
Точность (accuracy): 96%
F1-score: 0.91 (макро-усредненный)
Precision (точность) и Recall (полнота) для класса токсичных комментариев (1) — 0.79 и 0.89 соответственно, что говорит о хорошем балансе между точностью и полнотой.

**4. Выводы:**
Применение BERT эмбеддингов значительно улучшает результаты, особенно по сравнению с традиционными методами, такими как TF-IDF.
Оптимизация гиперпараметров с использованием Optuna позволяет найти лучшие настройки для моделей и улучшить их точность.
Random Forest показал себя как стабильная модель для данной задачи, однако методы на основе BERT показали лучшее качество и могли бы быть использованы для дальнейших улучшений.

**5. Рекомендации:**
Модели на основе BERT можно дополнительно дообучить на специфических данных или настроить на другие языки для улучшения универсальности.
Рассмотрение гибридных моделей с использованием нескольких подходов (например, объединение Random Forest и нейросетевых моделей).
Дальнейшая оптимизация и тестирование на новых данных для повышения устойчивости модели.
Проект показал хорошие результаты, и с использованием оптимизации гиперпараметров и BERT эмбеддингов удалось достичь высокой точности классификации токсичных комментариев! 